In [ ]:
from functools import reduce

import pandas as pd
import numpy as np
from sklearn.metrics import ndcg_score, average_precision_score

import clearml

from dotenv import load_dotenv
load_dotenv()

In [ ]:
data_path = 'data/'
train_data = pd.read_parquet(data_path + 'ratings_train.pq')
test_data = pd.read_parquet(data_path + 'ratings_test.pq')
groups5 = pd.read_parquet(data_path + 'groups5.pq')
groups6 = pd.read_parquet(data_path + 'groups6.pq')
groups7 = pd.read_parquet(data_path + 'groups7.pq')

In [ ]:
for i, group in enumerate([groups5, groups6, groups7]):
    test_data = test_data.merge(group, on='userId').rename(columns={'group': f'group{i+5}'})
test_data

In [ ]:
task = clearml.Task.init(
    project_name = 'MoviesGRS_MFDP', 
    task_name = 'PopularMoviesRecommender', 
    tags = ['Baseline','PopularMoviesRecommender']
)

In [ ]:
top_popular_films = (
    train_data
    .groupby(by='movieId')
    .agg({'userId': 'nunique'})
    .sort_values(by='userId', ascending=False)
    .rename(columns={'userId': 'userCount'})
    .reset_index()
)
top_popular_films

In [ ]:
top_popular_films_ids = top_popular_films.movieId.values

In [ ]:
users_watch_history_train = (
    train_data
    .groupby(by='userId')
    .agg({'movieId': list})
    .reset_index()
)
users_watch_history_train['unwatched'] = (
    users_watch_history_train.movieId
    .apply(lambda x: top_popular_films_ids[
        np.isin(top_popular_films_ids, x, invert=True)
    ])
)
users_watch_history_train

In [ ]:
recommendation = users_watch_history_train[['userId', 'unwatched']].rename(columns={'unwatched': 'recommendation'})
recommendation

In [ ]:
test_data

In [ ]:
users_watch_history_test: pd.DataFrame = (
    test_data
    .sort_values(by='rating', ascending=False)
    .groupby(by='userId')
    .agg(
        {
        **{f'group{i}': 'first' for i in range(5, 8)},
        'movieId': list
        }
    )
    .reset_index()
)
users_watch_history_test

In [ ]:
recom_and_history_test = users_watch_history_test.merge(recommendation, on='userId')
recom_and_history_test

In [ ]:
for group in [f'group{i}' for i in range(5, 8)]:
    group_recom = (
        recom_and_history_test
        .groupby(by=group)
        .agg({'recommendation': lambda x: reduce(np.intersect1d, x)})
        .reset_index()
    )

    def foo(x):
        result = []
        for movie in top_popular_films_ids:
            if movie in x:
                result.append(movie)
                if len(result) == 10:
                    return result

    group_recom['recommendation'] = (
        group_recom.recommendation
        .apply(foo)
    ) 

    group_recom = group_recom.rename(columns={'recommendation': f'{group}_rec'})

    recom_and_history_test = recom_and_history_test.merge(group_recom, on=group)

recom_and_history_test.drop(columns='recommendation', inplace=True)

In [ ]:
recom_and_history_test

In [ ]:
for group in [f'group{i}' for i in range(5, 8)]:
    rec_array = recommendation.values
    recom_and_history_test[f'{group}_relevance'] = (
        recom_and_history_test
        .apply(lambda row: np.isin(row[f'{group}_rec'], row['movieId']).astype(int), axis=1)
    )
recom_and_history_test

In [ ]:
for group in [f'group{i}' for i in range(5, 8)]:
    recom_and_history_test[f'{group}_P_k'] = (
        recom_and_history_test[f'{group}_relevance']
        .apply(lambda x: np.cumsum(x) * x / np.arange(1, len(x) + 1), 2)    
    )
    recom_and_history_test[f'{group}_P_k'] = (
        recom_and_history_test
        .apply(lambda row: row[f'{group}_P_k'].sum() / min(len(row['movieId']), len(row[f'{group}_rec'])), axis=1)
    )
    recom_and_history_test[f'{group}_P_k'] = (
        recom_and_history_test[f'{group}_P_k']
        .apply(lambda x: np.around(x, 2))
    )
recom_and_history_test

In [ ]:
logger = clearml.Logger.current_logger()

In [ ]:
metrics_results = {}

In [ ]:
for group in [f'group{i}' for i in range(5, 8)]:
    metrics_name = f"MAP_{group}"
    metrics_value = (
        recom_and_history_test
        .groupby(by=group)
        [f'{group}_P_k']
        .mean()
        .mean()
    )
    
    metrics_results[metrics_name] = metrics_value

In [ ]:
for group in [f'group{i}' for i in range(5, 8)]:    
    recommend_df = recom_and_history_test[['userId', f'{group}_rec']].explode(f'{group}_rec')
    recommend_df = (
        recommend_df
        .merge(
            test_data, 
            left_on=['userId', f'{group}_rec'],
            right_on=['userId', 'movieId'],
            how='left'    
        )
        [['userId', f'{group}_rec', 'rating']]
        .fillna(0)
    )
    recom_and_history_test[f'{group}_rec_ratings'] = (
        recommend_df
        .groupby(by='userId')
        .agg({'rating': list})
        .reset_index()
        .rating
    )
recom_and_history_test


In [ ]:
recom_and_history_test['pseudo_model_output'] = (
    recom_and_history_test[f'{group}_rec']
    .apply(lambda x: [(len(x) - i) for i in range(len(x))])
)
recom_and_history_test

In [ ]:
for group in [f'group{i}' for i in range(5, 8)]:
    recom_and_history_test[f'{group}_NDCG_k'] = (
        recom_and_history_test.apply(
            lambda row: ndcg_score([row[f'{group}_rec_ratings']], [row['pseudo_model_output']]),
            axis=1
        )
    )


In [ ]:
for group in [f'group{i}' for i in range(5, 8)]:
    metrics_name = f"NDCG_{group}"
    metrics_value = (
        recom_and_history_test
        .groupby(by=group)
        [f'{group}_NDCG_k']
        .mean()
        .mean()
    )
    
    metrics_results[metrics_name] = metrics_value

In [ ]:
metrics_results_2d = {}
for res in metrics_results:
    m, g = res.split('_')
    metrics_results_2d[m] = {**metrics_results_2d.get(m, {}), **{g: metrics_results[res]}}
pd.DataFrame(metrics_results_2d)

In [ ]:
task.upload_artifact('metrics_results_df', pd.DataFrame(metrics_results_2d))

In [ ]:
task.close()